In [1]:
import ipywidgets as widgets
from os import listdir


# Récupère le nom des images

In [10]:
import json
from os import listdir
import os

# Chemin du dossier contenant les images
dossier_images = "images"

# Liste pour stocker les noms des images
image_names = []

# Vérification si le dossier existe
if os.path.exists(dossier_images) and os.path.isdir(dossier_images):
    # Parcours des fichiers dans le dossier
    for fichier in os.listdir(dossier_images):
        # Vérification si le fichier est une image (extension .jpg, .png, .jpeg, etc.)
        if fichier.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Ajout du nom du fichier à la liste
            image_names.append(fichier)

# Limiter le nombre d'images à 30
image_names = image_names[:30]

# Construire la liste complète des chemins d'accès aux images
images = ["./Images/" + name for name in image_names]

In [11]:
images

['./Images/Image_150.jpg',
 './Images/Image_144.jpg',
 './Images/Image_29.jpg',
 './Images/Image_28.jpg',
 './Images/Image_14.jpg',
 './Images/Image_145.jpg',
 './Images/Image_16.jpg',
 './Images/Image_146.jpg',
 './Images/Image_142.jpg',
 './Images/Image_13.jpg',
 './Images/Image_12.jpg',
 './Images/Image_143.jpg',
 './Images/Image_141.jpg',
 './Images/Image_38.jpg',
 './Images/Image_10.jpg',
 './Images/Image_11.jpg',
 './Images/Image_39.jpg',
 './Images/Image_140.jpg',
 './Images/Image_133.jpg',
 './Images/Image_127.jpg',
 './Images/Image_76.jpg',
 './Images/Image_62.jpg',
 './Images/Image_89.jpg',
 './Images/Image_88.jpg',
 './Images/Image_63.jpg',
 './Images/Image_77.jpg',
 './Images/Image_126.jpg',
 './Images/Image_132.jpg',
 './Images/Image_130.jpg',
 './Images/Image_61.jpg']

# Interface pour sélectionner les images favorites

In [12]:
from ipywidgets import GridspecLayout, Image, interact
favorite_images = []

checkboxes = [widgets.Checkbox(value=False, description='Favorite') for _ in range(len(images))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(images), width='400px')
for i, (img, checkbox) in enumerate(zip(images, checkboxes)):
  file = open(img, "rb")
  image = file.read()
  image_widget = widgets.Image(
    value=image,
    format='png',
    width=100,
    height=100,
  )
  layout[i,0] = image_widget
  layout[i, 1] = checkbox

# Button to get selected images
button = widgets.Button(description="Select")

# Output widget to display selected images
output = widgets.Output()

# Function to get selected images
def get_selected_images(btn):
    global favorite_images
    favorite_images = []
    selected_paths = [images[i] for i, checkbox in enumerate(checkboxes) if checkbox.value]
    with output:
        output.clear_output()
        print("Selected Images:")
        for path in selected_paths:
            print(path)
            favorite_images.append(path)        

# Link button click event to function
button.on_click(get_selected_images)

# Display the layout and button
display(layout, button, output)

GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\…

Button(description='Select', style=ButtonStyle())

Output()

# Afficher les images favorites

In [16]:
layout = GridspecLayout(n_columns=3, n_rows=len(images)//3, width='400px')

for i, img in enumerate(favorite_images):
    with open(img, "rb") as file:
        image = file.read()
        
    image_widget = Image(
        value=image,
        format='png',
        width=100,
        height=100,
    )
    
    row = i // 3  # Divise i par 2 pour obtenir le numéro de la ligne
    col = i % 3   # Prend le reste de la division de i par 2 pour obtenir le numéro de la colonne
    
    layout[row, col] = image_widget

display(layout)


GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe1\x1a\xb4Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0f\x01…

# Algo calcul des préférences 

## Tags des images favorites

In [23]:
import json

favorite_images_without_extension = []

# Boucle pour retirer l'extension des noms d'images
for nom_image in favorite_images:
    nom_sans_extension, extension = os.path.splitext(nom_image)
    favorite_images_without_extension.append(nom_sans_extension)

file_names = [image.split("/")[-1] for image in favorite_images_without_extension]

json_file_path = "images_data.json"

with open(json_file_path, "r") as file:
    data_json = json.load(file)

data = []
color1 = None
color2 = None

# Récupérer les données pour chaque image dans file_names
for file_name in file_names:
    image_data = data_json.get(file_name)
    if image_data:
        # Extraire les informations nécessaires pour chaque image
        couleur_name = image_data.get("couleurs_predominante_name", [])
        if len(couleur_name) >= 1:
            color1 = couleur_name[0]
        if len(couleur_name) >= 2:
            color2 = couleur_name[1]
        orientation = image_data.get("orientation", "")
        data.append([color1,color2, orientation])

for item in data:
    print(item)



[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']
[None, None, 'paysage']


## Tag favori

In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Encode categorical features
label_encoders = [LabelEncoder() for _ in range(len(data[0]))]
encoded_data = []
for i, column in enumerate(zip(*data)):
    encoded_data.append(label_encoders[i].fit_transform(column))

X = list(zip(*encoded_data))  # Features

# Clustering
k = 2  # Number of clusters
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(X)
clusters = kmeans.labels_

# Add cluster labels to the original data
data_with_clusters = pd.DataFrame(data, columns=["Color", "Category", "Size", "Type"])
data_with_clusters["Cluster"] = clusters

# Recommendation function
def recommend_items(cluster, data_with_clusters):
    items_in_cluster = data_with_clusters[data_with_clusters["Cluster"] == cluster]
    recommended_items = items_in_cluster.sample(n=5)  # Sample 3 items from the cluster
    return recommended_items

# Example usage
user_interaction = ["green", "nature", "thumbnail", "landscape"]  # Assuming user interacted with this item
encoded_interaction = [label_encoders[i].transform([val])[0] for i, val in enumerate(user_interaction)]
cluster = kmeans.predict([encoded_interaction])[0]
recommendations = recommend_items(cluster, data_with_clusters)
print("Recommended items:")
print(recommendations)

NameError: name 'data' is not defined